In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [20]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/'
                      'machine-learning-databases/wine/wine.data',
                      header=None)

df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue',
                   'OD280/OD315 of diluted wines', 'Proline']

In [21]:
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, random_state=0)

In [22]:
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [23]:
# Unique class
print(np.unique(y_train))

[1 2 3]


In [24]:
mvs = []

# Calculate mean vectors for each class
for label in np.unique(y):
    mv = np.mean(X_train_std[y_train == label, :], axis=0)
    mvs.append(mv)
    mv_print = ", ".join(f"{i:5.2f}" for i in mv)
    print(f"Label={label}, MV:{mv_print}")

Label=1, MV: 0.93, -0.31,  0.26, -0.80,  0.30,  0.96,  1.05, -0.63,  0.54,  0.22,  0.49,  0.80,  1.20
Label=2, MV:-0.87, -0.39, -0.44,  0.25, -0.24, -0.11,  0.02, -0.02,  0.11, -0.88,  0.44,  0.28, -0.70
Label=3, MV: 0.16,  0.89,  0.32,  0.57, -0.01, -0.95, -1.23,  0.74, -0.77,  0.98, -1.17, -1.30, -0.39


In [25]:
d = X_train_std.shape[1]

# Within-class scatter matrix
S_W = np.zeros((d, d))

for label, mv in zip(np.unique(y), mvs):
    # Individual scatter matrix for each class
    S_i = np.zeros((d, d))
    for row in X_train_std[y_train == label]:
        row = row.reshape(d,1)
        mv = mv.reshape(d,1)
        rmmv = row - mv
        S_i +=  rmmv @ rmmv.T
    S_W += S_i

In [26]:
# Within-class scatter matrix
print(S_W)

[[ 51.45101306   0.670318    -2.06258511  -3.47238501   4.84388869
    9.0901686    5.68687116  -2.41246623   1.10812187  18.65670392
    1.62444216  -1.12474241   5.56659679]
 [  0.670318    84.99305482   9.47473115  14.83445274  -7.89867542
    2.51390052   1.03042395  13.69191016   8.99868657  -6.65578513
  -16.78460821   7.3565464  -11.07769284]
 [ -2.06258511   9.47473115 107.97264694  62.55949709  14.26093065
   13.5860753   11.00744636  20.67291239   2.32277049  -2.34240043
    4.37462302  10.31171475  -3.64807231]
 [ -3.47238501  14.83445274  62.55949709  84.24878958   1.85151117
    5.27001803   6.27341911  14.78290279  -0.76047476  -5.43540683
    2.20501071  12.29673823  -4.9820588 ]
 [  4.84388869  -7.89867542  14.26093065   1.85151117 117.45959015
    7.05131981   4.79572529 -30.10855865  28.00924022   2.78008086
    6.00007813  -4.91405093  15.67292609]
 [  9.0901686    2.51390052  13.5860753    5.27001803   7.05131981
   54.9426362   27.24579247  -6.96515569  30.77187279

In [33]:
d = X_train_std.shape[1]

# Scaled within-class scatter matrix
S_W = np.zeros((d, d))

for label in np.unique(y):
    S_i = np.cov(X_train_std[y_train == label], rowvar=False)
    S_W += S_i
print(S_W)

[[ 1.24583230e+00  5.13845419e-02 -3.29458743e-02 -8.95327553e-02
   1.15462644e-01  2.45386077e-01  1.50110340e-01 -5.56872417e-02
   9.90787667e-02  4.87857619e-01  2.37631203e-02 -6.97155580e-03
   1.34211228e-01]
 [ 5.13845419e-02  2.15469734e+00  2.24341335e-01  3.54339113e-01
  -1.96603001e-01  3.60539531e-02 -5.40272908e-03  3.40166629e-01
   1.68397895e-01 -1.67482338e-01 -3.57550070e-01  1.77277407e-01
  -2.54171698e-01]
 [-3.29458743e-02  2.24341335e-01  2.52503549e+00  1.45812968e+00
   3.70291335e-01  3.36648199e-01  2.38744163e-01  4.71351788e-01
   3.09124828e-02 -5.64161940e-02  1.28417016e-01  2.34080150e-01
  -1.06647806e-01]
 [-8.95327553e-02  3.54339113e-01  1.45812968e+00  1.98052354e+00
   8.13974218e-02  1.17951658e-01  1.25215529e-01  3.27635494e-01
  -3.73250441e-02 -1.29452248e-01  6.16221030e-02  2.55451847e-01
  -1.29087470e-01]
 [ 1.15462644e-01 -1.96603001e-01  3.70291335e-01  8.13974218e-02
   2.70477156e+00  1.60415112e-01  1.49801537e-01 -7.36919038e-01


In [34]:
d = X_train_std.shape[1] # number of features

# Mean overal
mva = np.mean(X_train_std, axis=0).reshape(d,1) # make column vector

S_B = np.zeros((d, d))
for label, mv in zip(np.unique(y), mvs):
    n = X_train[y_train == label, :].shape[0]
    mv = mv.reshape(d, 1)  # make column vector
    mmm = mv - mva
    S_B += n * mmm @ mmm.T
print(S_B)

[[ 72.54898694  10.15085406  30.43493641 -36.95537717  21.49817241
   34.66924776  31.10545402 -18.39143902  10.7602395   51.40688925
   -7.50573953  10.23103794  72.26519802]
 [ 10.15085406  39.00694518  15.32718585  22.87506273   0.48064793
  -39.56671878 -51.73245374  31.3464995  -32.60256622  44.47726853
  -50.85654449 -55.76044251 -13.83461128]
 [ 30.43493641  15.32718585  16.02735306  -7.24397229   8.27440794
    1.46362979  -3.4672309    2.2735902   -5.53040633  32.54548762
  -17.81608831 -12.5503173   23.26415138]
 [-36.95537717  22.87506273  -7.24397229  39.75121042 -12.83663892
  -50.8027022  -57.69290912  34.67799011 -30.93129535   1.63442966
  -33.34032298 -47.8860156  -54.67824413]
 [ 21.49817241   0.48064793   8.27440794 -12.83663892   6.54040985
   13.26003311  12.98847983  -7.73062353   5.48196174  12.72620954
    1.12481807   6.87729589  23.02418448]
 [ 34.66924776 -39.56671878   1.46362979 -50.8027022   13.26003311
   69.0573638   81.14176016 -48.8730331   45.44884096

In [35]:
eigen_vals, eigen_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

In [36]:
eigen_vals

array([-2.84217094e-14+0.00000000e+00j,  1.56436361e+02+0.00000000e+00j,
        4.52721581e+02+0.00000000e+00j,  4.06692903e-14+0.00000000e+00j,
        8.93640010e-15+2.80307206e-14j,  8.93640010e-15-2.80307206e-14j,
       -2.95059953e-14+1.56519907e-14j, -2.95059953e-14-1.56519907e-14j,
       -2.18183060e-14+0.00000000e+00j,  2.08601836e-14+0.00000000e+00j,
        4.07252608e-15+0.00000000e+00j,  4.28948526e-18+4.93174010e-15j,
        4.28948526e-18-4.93174010e-15j])

In [37]:
# Make a list of (eigenvalue, eigenvector) tuples
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:, i])
               for i in range(len(eigen_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eigen_pairs = sorted(eigen_pairs, key=lambda k: k[0], reverse=True)

# Visually confirm that the list is correctly sorted by decreasing eigenvalues

print('Eigenvalues in decreasing order:\n')
for eigen_val in eigen_pairs:
    print(eigen_val[0])

Eigenvalues in decreasing order:

452.7215812449743
156.43636121952318
4.066929029269564e-14
3.340042769632176e-14
3.340042769632176e-14
2.9420750270010086e-14
2.9420750270010086e-14
2.842170943040401e-14
2.181830602025924e-14
2.086018357074801e-14
4.931741968470871e-15
4.931741968470871e-15
4.072526084406939e-15


In [43]:
aa = eigen_pairs[0][1].reshape(-1,1)

In [45]:
aaa = np.hstack((aa,aa))

In [48]:
np.abs(X_train_std.dot(aaa))

array([[1.92942894e+00, 1.92942894e+00],
       [1.57807512e-01, 1.57807512e-01],
       [1.71629527e+00, 1.71629527e+00],
       [1.28405614e+00, 1.28405614e+00],
       [3.18775399e-01, 3.18775399e-01],
       [1.27250409e+00, 1.27250409e+00],
       [1.85236273e+00, 1.85236273e+00],
       [1.02210223e+00, 1.02210223e+00],
       [6.88039360e-01, 6.88039360e-01],
       [1.48032326e+00, 1.48032326e+00],
       [3.36464162e-01, 3.36464162e-01],
       [6.77346281e-01, 6.77346281e-01],
       [1.16249806e+00, 1.16249806e+00],
       [1.29307372e-01, 1.29307372e-01],
       [1.40601187e+00, 1.40601187e+00],
       [8.97318514e-01, 8.97318514e-01],
       [3.52022450e-01, 3.52022450e-01],
       [9.30409494e-01, 9.30409494e-01],
       [5.59207624e-01, 5.59207624e-01],
       [1.91287339e+00, 1.91287339e+00],
       [7.77419710e-01, 7.77419710e-01],
       [6.46820433e-01, 6.46820433e-01],
       [1.93700879e+00, 1.93700879e+00],
       [2.10316686e+00, 2.10316686e+00],
       [4.348430